In [1]:
import pathlib
import os, sys
import numpy as np
from loguru import logger
from collections import defaultdict
from openset_imagenet.util import ccr_at_fpr
import openset_imagenet
from matplotlib import pyplot

In [2]:
def load_scores(args):
    # we sort them as follows: protocol, loss, algorithm
    scores = defaultdict(lambda: defaultdict(dict))
    ground_truths = {}
    for net in args["networks"]:
                output_directory = pathlib.Path(args["output_directory"])
                score_file = f"{net}.npz"
                if os.path.exists(score_file):
                    # remember files
                    results = np.load(score_file)
                    scores[net] = results["scores"] # only change in maxlogits case

                    if len(ground_truths) == 0:
                        ground_truths = results["gt"].astype(int)
                    else:
                        assert np.all(results["gt"] == ground_truths)

                    logger.info(f"Loaded score file {score_file} for net {net}")
                else:
                    logger.warning(f"Did not find score file {score_file} for net {net}")

    return scores, ground_truths

In [3]:
arguments = { 
  "output_directory": "experiments/ex_5",
  "networks": ["net_4", "net_5", "net_6", "net_7_ep_50", "net_11"]
  }
THRESHOLDS = {
              1e-3: "$10^{-3}$",
              1e-2: "$10^{-2}$",
              1e-1: "$10^{-1}$",
              1: "$1$",
}

scores, ground_truths = load_scores(arguments)
# we get ccr@fpr for each network
for net in arguments["networks"]:
    ccrs = ccr_at_fpr(ground_truths, scores[net], THRESHOLDS)
    print(f"Network {net}: {ccrs} for thresholds {THRESHOLDS.values()}")

2024-05-19 08:57:08.671 | INFO     | __main__:load_scores:18 - Loaded score file net_4.npz for net net_4
2024-05-19 08:57:08.680 | INFO     | __main__:load_scores:18 - Loaded score file net_5.npz for net net_5
2024-05-19 08:57:08.687 | INFO     | __main__:load_scores:18 - Loaded score file net_6.npz for net net_6
2024-05-19 08:57:08.714 | INFO     | __main__:load_scores:18 - Loaded score file net_7_ep_50.npz for net net_7_ep_50
2024-05-19 08:57:08.719 | INFO     | __main__:load_scores:18 - Loaded score file net_11.npz for net net_11


Network net_4: [0.0006666666666666666, 0.013333333333333334, 0.04466666666666667, 0.16333333333333333] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_5: [0.05266666666666667, 0.09866666666666667, 0.278, 0.528] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_6: [None, 0.11133333333333334, 0.2633333333333333, 0.546] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_7_ep_50: [0.009333333333333334, 0.11133333333333334, 0.30533333333333335, 0.616] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
Network net_11: [0.016, 0.12333333333333334, 0.3626666666666667, 0.6526666666666666] for thresholds dict_values(['$10^{-3}$', '$10^{-2}$', '$10^{-1}$', '$1$'])
